In [ ]:
from thruster_allocator import thruster_positions, thruster_directions, ThrustAllocator, thrust_map
from controls_core.params import IMU_ZERO
from attitude_control import AttitudeControl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp

import random

%reload_ext autoreload
%autoreload 2

# Thrust Allocator Test

## Thruster Positions and Directions

In [ ]:
%matplotlib qt
ax = plt.figure().add_subplot(projection="3d")
ax.quiver3D(*thruster_positions.T, *thruster_directions.T, length=0.1)
ax.scatter(*(0,0,0))
ax.set_aspect('equal')

## Movement Test

### Thrust Newtons

In [ ]:
%matplotlib qt
thrustAllocator = ThrustAllocator()
outputThrusts = thrustAllocator.getThrusts(
    linear_accelerations = [1.0, 0, 0],
    angular_accelerations = [0, 0, 0]
)

ax = plt.figure().add_subplot(projection="3d")
ax.quiver3D(*thruster_positions.T, *(outputThrusts[:, None] * thruster_directions).T, length=0.02)
ax.scatter(*(0,0,0))
ax.set_aspect('equal')

### Thrust PWMs

In [ ]:
pwms = thrustAllocator.getThrustPWMs(
    linear_accelerations = [0, 0, -3.0],
    angular_accelerations = [0, 0, 0]
)

pwms[np.array([3, 2, 5, 1, 4, 0])]

# Thrust Map

In [ ]:
%matplotlib inline
thrust_newtons = thrust_map[:,0]
thrust_pwm = thrust_map[:,1]
plt.plot(thrust_newtons, thrust_pwm)
plt.xlabel("thrust_newtons")
plt.ylabel("thrust_pwm")
plt.show()
plt.plot(thrust_pwm, thrust_newtons)
plt.xlabel("thrust_pwm")
plt.ylabel("thrust_newtons")
plt.show()

In [ ]:
thrust_converted = thrust_map[
    np.searchsorted(thrust_pwm, 172, side="left"), 0
]
thrust_converted

# Attitude Control

In [ ]:
time_interval = 0.1

startpoint = -np.pi/2

attitudeControl = AttitudeControl()

def eval_accel(t, u):
    y, v = u
    thrust = attitudeControl.getAttitudeCorrection(currAttRPY=[0,0,y], targetAttRPY=[0,0,0])[2]
    accel = thrust + random.normalvariate(0, 0.1)
    return [v, accel]

def constrain(val, low, high):
    return min(high, max(low, val))

t_span = [0,16]

sol = solve_ivp(eval_accel, t_span=t_span, y0=[startpoint,0], atol=10, max_step=time_interval)
angles = list(map(attitudeControl.boundAngle, sol.y[0]))

plt.plot(sol.t, angles)
plt.axhline(IMU_ZERO[2], color='k')
plt.axhline(IMU_ZERO[2]+0.2, color='r', linestyle='--')
plt.axhline(IMU_ZERO[2]-0.2, color='r', linestyle='--')